In [1]:
import pandas as pd
import numpy as np
import json
import string

In [2]:
with open('../Files/words_dictionary.json', 'r') as f:
    dic = json.load(f)

In [3]:
dic=list(dic.keys())

In [4]:
ej=dic[20]
ej

'aaronical'

In [5]:
total= dict.fromkeys(string.ascii_lowercase, 0)
total["-"]=0
total

{'a': 0,
 'b': 0,
 'c': 0,
 'd': 0,
 'e': 0,
 'f': 0,
 'g': 0,
 'h': 0,
 'i': 0,
 'j': 0,
 'k': 0,
 'l': 0,
 'm': 0,
 'n': 0,
 'o': 0,
 'p': 0,
 'q': 0,
 'r': 0,
 's': 0,
 't': 0,
 'u': 0,
 'v': 0,
 'w': 0,
 'x': 0,
 'y': 0,
 'z': 0,
 '-': 0}

In [6]:
df=pd.DataFrame.from_dict(total, orient='index')
df.head()

,0
a,0
b,0
c,0
d,0
e,0


In [7]:
list_total=list(total)
for i in list_total:
    df[i]=0
df.drop([0], axis=1, inplace=True)

In [8]:
df.head()

,a,b,c,d,e,f,g,h,i,j,...,r,s,t,u,v,w,x,y,z,-
a,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
b,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
c,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
d,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
e,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
for i in dic: 
        for j in set(i):
            for x in i:
                df.loc[j][x]+=1
df.head(20)

,a,b,c,d,e,f,g,h,i,j,...,r,s,t,u,v,w,x,y,z,-
a,295791,40674,99166,65076,202332,19561,49122,57254,183593,3071,...,150740,136801,149544,67820,18945,11300,6185,41998,9455,2
b,52241,63940,18802,16185,60872,4350,9892,10991,47675,886,...,38874,37146,30428,25805,3585,3019,1209,10518,2167,0
c,115802,17475,152979,33390,129131,10797,19248,39499,124607,1591,...,87594,80768,85696,46583,9788,3012,3056,27940,3604,1
d,76427,15481,34550,113189,119256,9487,18005,20752,87495,1317,...,64307,57597,49669,37180,7763,6958,2284,16623,4051,0
e,182616,43575,102160,87612,376457,26991,51517,59728,201037,3323,...,186121,180341,165917,86472,27422,14747,8114,40294,10990,2
f,21805,4155,10881,9525,36285,39238,6570,5212,32665,229,...,26226,21309,18552,15377,1990,2152,692,5324,900,0
g,60919,9775,20683,18282,67071,6719,82625,18363,74791,931,...,48737,41340,39361,25131,5300,4058,1141,14924,3203,0
h,69928,10850,42728,21121,79200,5406,17529,92370,66520,457,...,55202,56276,53402,21529,2837,5899,1997,21703,2882,1
i,178422,36199,104526,68736,214636,24306,58736,56176,313009,2825,...,147941,158941,156048,72268,23546,9843,6829,41168,11698,2
j,4075,977,1774,1456,4663,248,1086,476,3625,5458,...,2379,2570,2396,2745,403,239,46,785,174,2


In [15]:
df.to_csv("../Files/dataFrame_letters.csv")

In [2]:
df=pd.read_csv("../Files/dataFrame_letters.csv", index_col=0)
df.head()

,a,b,c,d,e,f,g,h,i,j,...,r,s,t,u,v,w,x,y,z,-
a,295791,40674,99166,65076,202332,19561,49122,57254,183593,3071,...,150740,136801,149544,67820,18945,11300,6185,41998,9455,2
b,52241,63940,18802,16185,60872,4350,9892,10991,47675,886,...,38874,37146,30428,25805,3585,3019,1209,10518,2167,0
c,115802,17475,152979,33390,129131,10797,19248,39499,124607,1591,...,87594,80768,85696,46583,9788,3012,3056,27940,3604,1
d,76427,15481,34550,113189,119256,9487,18005,20752,87495,1317,...,64307,57597,49669,37180,7763,6958,2284,16623,4051,0
e,182616,43575,102160,87612,376457,26991,51517,59728,201037,3323,...,186121,180341,165917,86472,27422,14747,8114,40294,10990,2


In [3]:
df_per= (df.div(df.sum(axis=1), axis=0)*100).round(2).astype(str) + '%'
df_per.head()

,a,b,c,d,e,f,g,h,i,j,...,r,s,t,u,v,w,x,y,z,-
a,13.57%,1.87%,4.55%,2.98%,9.28%,0.9%,2.25%,2.63%,8.42%,0.14%,...,6.91%,6.27%,6.86%,3.11%,0.87%,0.52%,0.28%,1.93%,0.43%,0.0%
b,9.05%,11.08%,3.26%,2.8%,10.55%,0.75%,1.71%,1.9%,8.26%,0.15%,...,6.74%,6.44%,5.27%,4.47%,0.62%,0.52%,0.21%,1.82%,0.38%,0.0%
c,8.59%,1.3%,11.35%,2.48%,9.58%,0.8%,1.43%,2.93%,9.25%,0.12%,...,6.5%,5.99%,6.36%,3.46%,0.73%,0.22%,0.23%,2.07%,0.27%,0.0%
d,7.8%,1.58%,3.53%,11.55%,12.17%,0.97%,1.84%,2.12%,8.93%,0.13%,...,6.56%,5.88%,5.07%,3.79%,0.79%,0.71%,0.23%,1.7%,0.41%,0.0%
e,7.33%,1.75%,4.1%,3.51%,15.1%,1.08%,2.07%,2.4%,8.06%,0.13%,...,7.47%,7.23%,6.66%,3.47%,1.1%,0.59%,0.33%,1.62%,0.44%,0.0%


# Calculate probability given 2 or more letters

In [4]:
li=["a", "b"]

In [5]:
df_cal= df.loc[li]
df_cal

,a,b,c,d,e,f,g,h,i,j,...,r,s,t,u,v,w,x,y,z,-
a,295791,40674,99166,65076,202332,19561,49122,57254,183593,3071,...,150740,136801,149544,67820,18945,11300,6185,41998,9455,2
b,52241,63940,18802,16185,60872,4350,9892,10991,47675,886,...,38874,37146,30428,25805,3585,3019,1209,10518,2167,0


In [6]:
total = df_cal.sum()
total.name = 'total'
# Assign sum of all rows of DataFrame as a new Row
df_cal = df_cal.append(total.transpose())
df_cal

,a,b,c,d,e,f,g,h,i,j,...,r,s,t,u,v,w,x,y,z,-
a,295791,40674,99166,65076,202332,19561,49122,57254,183593,3071,...,150740,136801,149544,67820,18945,11300,6185,41998,9455,2
b,52241,63940,18802,16185,60872,4350,9892,10991,47675,886,...,38874,37146,30428,25805,3585,3019,1209,10518,2167,0
total,348032,104614,117968,81261,263204,23911,59014,68245,231268,3957,...,189614,173947,179972,93625,22530,14319,7394,52516,11622,2


In [7]:
df_cal.drop(li, axis=0, inplace=True)
df_cal.drop(li, axis=1, inplace=True)
df_cal.head()

,c,d,e,f,g,h,i,j,k,l,...,r,s,t,u,v,w,x,y,z,-
total,117968,81261,263204,23911,59014,68245,231268,3957,18971,167214,...,189614,173947,179972,93625,22530,14319,7394,52516,11622,2


In [8]:
df_cal= df_cal.T[["total"]]
df_cal['per'] = (df_cal.total * 100 / df_cal.total.sum()).round(2).astype(str) + '%'
df_cal.head()

,total,per
c,117968,5.12%
d,81261,3.53%
e,263204,11.42%
f,23911,1.04%
g,59014,2.56%


In [9]:
df_cal.sort_values(by="total", ascending=False, inplace= True)
df_cal.head()

,total,per
e,263204,11.42%
i,231268,10.03%
n,190007,8.24%
r,189614,8.23%
t,179972,7.81%


In [10]:
df_cal.total.idxmax()

'e'

In [11]:
#NoVowel

def next_letter(*letter):
    
    #Create df with letters as an index
    df_words= df.loc[letter]
    #Get new row with total letters appears
    total = df_words.sum()
    total.name = 'total'
    # Assign sum of all rows of DataFrame as a new Row
    df_words = df_words.append(total.transpose())
    vowels=["a","e", "i", "o", "u"]
    #Delete vowels and letters from columns and rows
    remove= list(set(vowels+list(*letter)))
    df_words.drop(*letter, axis=0, inplace=True)
    df_words.drop(remove, axis=1, inplace=True)
    #New column with percentajes for each vowel
    df_words= df_words.T[["total"]]
    df_words['per'] = (df_words.total * 100 / df_words.total.sum()).round(4)
    df_words.sort_values(by="total", ascending=False, inplace= True)
    
    #Get recommended letter
    final_word= df_words.total.idxmax()
    print(df_words.head())
    if (df_words.iloc[0].per - df_words.iloc[1].per) < 1:
        return f'You should choose {final_word} with {df_words.loc[final_word].per.round(3)}% appear or {df_words.iloc[1].name} with {df_words.loc[df_words.iloc[1].name].per.round(3)}% appear'
    else:
        return f'You should choose {final_word} with a {df_words.loc[final_word].per.round(3)}% appear'

In [12]:
letters=["d", "c"]
next_letter(letters)

    total      per
n  159372  14.7008
r  151901  14.0117
s  138365  12.7631
t  135365  12.4864
l  116969  10.7895


'You should choose n with 14.701% appear or r with 14.012% appear'

In [57]:
#Vowel

def next_vowel(*vowel):
    
    #Create df with letters as an index
    df_vowel= df.loc[vowel]
    print(vowel)
    #Get new row with total letters appears    
    total = df_vowel.sum()
    total.name = 'total'
    # Assign sum of all rows of DataFrame as a new Row
    vowels=["a","e", "i", "o", "u"]
    vowels= [i for i in vowels if i not in list(*vowel)]
    df_vowel = df_vowel.append(total.transpose())
    df_vowel = df_vowel[[*vowels]].T[["total"]]
    #New column with percentajes for each vowel
    df_vowel['per'] = (df_vowel.total * 100 / df_vowel.total.sum()).round(4)
    df_vowel.sort_values(by="total", ascending=False, inplace= True)
    
    #Get recommended vowel
    final_vowel= df_vowel.total.idxmax()
    print(df_vowel.head())
    if (df_vowel.iloc[0].per - df_vowel.iloc[1].per) < 1:
        return f'You should choose {final_vowel} with {df_vowel.loc[final_vowel].per.round(3)}% appear or {df_vowel.iloc[1].name} with {df_vowel.loc[df_vowel.iloc[1].name].per.round(3)}% appear'
    else:
        return f'You should choose {final_vowel} with a {df_vowel.loc[final_vowel].per.round(3)}% appear'

In [60]:
letters=["d", "c", "e", "o"]
next_vowel(letters)

(['d', 'c', 'e', 'o'],)
    total      per
i  567603  43.4227
a  507824  38.8495
u  231730  17.7278


'You should choose i with a 43.423% appear'